This notebook is responsible for visualizing the data from the Github API, which should be provided in a serialized format.

It must be possible to run this notebook using jupyterlite.

If you do not see the newest version of the notebook, you have to "delete" it in your jupyterlite instance, as your local version is overriding the remote version provided by the deployment.

(jupyterlite) We need to manually install dependencies for the front-end, even though they are also defined in the `requirements.txt`.

TODO: Fixed versions to prevent version-drift.

In [ ]:
%pip install plotly 

In [ ]:
%pip install matplotlib

In [ ]:
%pip install ipywidgets

In [ ]:
import json
import datetime
import numpy as np
import plotly
import ipywidgets

In [ ]:
with open('commit_dates.json') as f:
	commit_dates = json.load(f)
	
# Convert all the dates to datetime objects
commit_dates = [datetime.datetime.strptime(date, '%Y-%m-%d %H:%M:%S') for date in commit_dates]

In [ ]:
# Use an ipywidget to set a start and end datetime
start_date = ipywidgets.DatePicker(description='Start Date')
end_date = ipywidgets.DatePicker(description='End Date')
display(start_date, end_date)

In [ ]:
# Plot a heatmap for the number of commits per day and hour of the week
def plot_heatmap(commit_dates, start_date=None, end_date=None):
	print(start_date)
	print(end_date)
	# Convert date objects to datetime objects
	if start_date is not None:
		start_date = datetime.datetime.combine(start_date, datetime.datetime.min.time())
	if end_date is not None:
		end_date = datetime.datetime.combine(end_date, datetime.datetime.max.time())
	# Filter the dates to only include those between the start and end dates
	# The dates are already sorted, so we can just filter the beginning and end
	if start_date is not None:
		commit_dates = [date for date in commit_dates if date >= start_date]
	if end_date is not None:
		commit_dates = [date for date in commit_dates if date <= end_date]
	
	# Get the day and hour of each commit
	days = [date.weekday() for date in commit_dates]
	y_vals = [date.hour for date in commit_dates]
	
	# Create a 2D histogram of the day and hour
	heatmap, xedges, yedges = np.histogram2d(days, y_vals, bins=(7, 24))
	
	# Create a plotly figure
	fig = plotly.graph_objs.FigureWidget()
	
	# Add a heatmap trace
	fig.add_trace(plotly.graph_objs.Heatmap(
		z=heatmap,
		x=xedges,
		y=yedges,
		colorscale='oranges',
		colorbar=dict(
			title='Commits'
		),
		hoverinfo='z'
	))
	
	# Add axis labels
	fig.update_layout(
		xaxis=dict(
			title='Hour of the day',
			tickmode='array',
			tickvals=np.arange(0, 24, 2),
			ticktext=np.arange(0, 24, 2)
		),
		yaxis=dict(
			title='Day of the week',
			tickmode='array',
			tickvals=np.arange(0, 7),
			ticktext=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
		)
	)
	
	return fig

plot_heatmap(commit_dates, start_date.value, end_date.value)